courier_detailed_trajectory.csv   ┐
                                  ├─► Feature extraction & aggregation
roads.csv                          │
                                  ▼
                     (causal features per region/hour/courier)
                                  │
                                  ▼
                  delivery_chongquin.csv  ← FINAL TRAINING TABLE


In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

In [2]:
orders = pd.read_csv("../LaDe/delivery/delivery_cq.csv", parse_dates=[
    "accept_time", "accept_gps_time",
    "delivery_time", "delivery_gps_time"
])
# orders["date"] = orders["ds"].apply(parse_ds)
orders.head()

/tmp/ipykernel_38981/544110044.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  orders = pd.read_csv("../LaDe/delivery/delivery_cq.csv", parse_dates=[
/tmp/ipykernel_38981/544110044.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  orders = pd.read_csv("../LaDe/delivery/delivery_cq.csv", parse_dates=[
/tmp/ipykernel_38981/544110044.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  orders = pd.read_csv("../LaDe/delivery/delivery_cq.csv", parse_dates=[
/tmp/ipykernel_38981/544110044.py:1: UserWarning: Could not infer format, so each element will be parsed individually,

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,accept_gps_lng,accept_gps_lat,delivery_time,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds
0,2031782,10,Chongqing,73,108.71571,30.90228,50,14,10-22 10:26:00,10-22 10:26:00,108.71826,30.95587,10-22 17:04:00,10-22 17:04:00,108.66361,30.96702,1022
1,4285071,10,Chongqing,3605,108.71639,30.90269,50,14,09-07 10:13:00,09-07 10:13:00,108.71791,30.95635,09-09 15:44:00,09-09 15:44:00,108.71644,30.90266,907
2,4056800,10,Chongqing,3605,108.71645,30.90259,50,14,06-26 09:49:00,06-26 09:49:00,108.71798,30.95635,06-27 16:03:00,06-27 16:03:00,108.71647,30.90251,626
3,3589481,10,Chongqing,3605,108.71650,30.90347,50,14,09-11 11:01:00,09-11 11:01:00,108.71823,30.95596,09-13 17:14:00,09-13 17:14:00,108.71650,30.90341,911
4,2752329,10,Chongqing,3605,108.71608,30.90409,50,14,10-01 09:52:00,10-01 09:52:00,108.71820,30.95598,10-01 18:30:00,10-01 18:30:00,108.71413,30.90397,1001


In [3]:
def parse_ds(ds, year=2021):
    ds = int(ds)
    month = ds // 100
    day = ds % 100
    return pd.Timestamp(year=year, month=month, day=day)

parse_ds(911)

Timestamp('2021-09-11 00:00:00')

In [4]:
# traj = pd.read_csv("../LaDe/courier_detailed_trajectory/courier_detailed_trajectory/.csv", parse_dates=["gps_time"])
traj20s = pd.read_pickle("../LaDe/data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz")


traj20s.head()

,ds,postman_id,gps_time,lat,lng
0,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:31:58,3.953700e+06,3.053400e+06
1,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:18,3.953700e+06,3.053398e+06
2,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:41,3.953700e+06,3.053398e+06
3,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:55:51,3.953700e+06,3.053398e+06
4,321,106f5ac22cfd1574b196d16fed62f90d,03-21 08:31:42,3.953929e+06,3.052367e+06


In [5]:
roads = pd.read_csv("../processed/roads_translated.csv")
roads.head(3)

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,city,geometry,city_chinese
0,4296533,5141,service,NaN,NaN,B,0,0,F,F,Hangzhou,LINESTRING (13373499.140421594 3538916.2947984...,杭州市
1,4296592,5115,tertiary,朝晖路,NaN,B,0,0,F,F,Hangzhou,LINESTRING (13376291.100042382 3539415.0794264...,杭州市
2,4298707,5114,secondary,体育场路,NaN,F,0,0,F,F,Hangzhou,LINESTRING (13375063.92510783 3538697.18123532...,杭州市


In [10]:
def haversine(lat1, lon1, lat2, lon2):
    ''' Distance between 2 lat long on a spherical surface '''
    R = 6371000
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))


def haversine_vec(lat1, lon1, lat2, lon2):
    """
    Vectorized haversine distance (meters)
    lat/lon inputs can be pandas Series or numpy arrays
    """
    R = 6371000.0  # Earth radius in meters

    lat1 = np.deg2rad(lat1)
    lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2)
    lon2 = np.deg2rad(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    )

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [7]:
traj20s = traj20s.sort_values(["postman_id", "gps_time"])
# traj20s.head(10)

,ds,postman_id,gps_time,lat,lng
37652971,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:00:02,1.708151e+06,1.064734e+06
37652972,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:00:32,1.708151e+06,1.064734e+06
37652973,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:01:02,1.708151e+06,1.064734e+06
37652974,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:01:32,1.708151e+06,1.064734e+06
37652975,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:02:02,1.708151e+06,1.064734e+06
37652976,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:02:32,1.708151e+06,1.064734e+06
37652977,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:03:17,1.708151e+06,1.064734e+06
37652978,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:03:47,1.708151e+06,1.064734e+06
37652979,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:04:17,1.708151e+06,1.064734e+06
37652980,321,0008c2b6a2314db8715301b7eeeebc5a,03-21 00:05:02,1.708151e+06,1.064734e+06


In [9]:
traj20s["dist"] = haversine_vec(
    traj20s["lat"].shift(), traj20s["lng"].shift(),
    traj20s["lat"], traj20s["lng"]
)

TypeError: cannot convert the series to <class 'float'>